In [ ]:
import os, sys
import pandas as pd
import numpy as np
import params as pm
import datetime as dt
import json
import pickle
import tqdm

# Select cohorts

In [ ]:
def gather_cohort(adults=True, patient_weight=True, icd_diagnoses=[], min_los=1, max_los=8, verbose=False):
    cohort = pd.read_csv("data/mimic-iv-1.0/icu/icustays.csv")
    # Filter for adults with icustay length between 1 and 8 days
    cohort = cohort[cohort['los'] >= min_los]
    cohort = cohort[cohort['los'] <= max_los]
    cohort = cohort[['subject_id', 'stay_id', 'hadm_id', 'intime', 'outtime', 'los']]

    ages = pd.read_csv("data/mimic-iv-1.0/core/patients.csv")
    ages = ages[ages['anchor_age'] >= 18]
    ages = ages[['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group']]

    admissions = pd.merge(cohort, ages, on=['subject_id'])

    # Admissions + Age information
    admissions = pd.merge(admissions, ages, on=['subject_id', 'gender', 'anchor_age'], how='inner')
    admissions
    if patient_weight:
        if verbose: print("Adding information about patientweight")
        # Add patient weight information
        weights = pd.read_csv("data/mimic-iv-1.0/icu/procedureevents.csv")
        weights = weights[['stay_id', 'hadm_id', 'patientweight']]
        admissions = pd.merge(admissions, weights, on=['hadm_id', 'stay_id']).drop_duplicates()
    admissions
    
    if len(icd_diagnoses) > 0:
        if verbose: print("Filtering for ICD diagnoses")
        diagnoses_icd = pd.read_csv("data/mimic-iv-1.0/hosp/diagnoses_icd.csv")
        diagnoses_icd = diagnoses_icd[diagnoses_icd[['icd_code', 'icd_version']].apply(tuple, axis=1).isin(icd_diagnoses)]
        diagnoses = diagnoses_icd[['hadm_id', 'icd_code', 'icd_version']]
        admissions = pd.merge(admissions, diagnoses, on=['hadm_id']).drop_duplicates()

    hadm_ids = admissions.hadm_id.unique()

    return hadm_ids, admissions

# Gather dataframes with basic vitals + meds + labs for each cohort

In [ ]:
'''
Gathers all ICU level medications. 
'''
def gather_meds(meds_dict, verbose=False):
    meds_ids = []
    for key in meds_dict:
        names = meds_dict[key]
        meds_ids += names

    if verbose: print("Gathering meds")    
    meds = pd.read_csv("data/mimic-iv-1.0/icu/inputevents.csv")
    meds = meds[meds['itemid'].isin(meds_ids)]
    meds = meds[meds['amount'] > 0]
    meds = meds[meds['amount'] < 9999]
    
    if verbose: print("Saving dataframes")
    meds = meds.drop_duplicates()
    meds.to_pickle('meds')
    return meds

'''
Gather all vitals records. 
'''
def gather_vitals(vitals_dict, verbose=False):
    vitals_itemids = []
    for key in vitals_dict:
        itemid = vitals_dict[key]
        vitals_itemids.extend(itemid)
    if verbose: print("Gathering vitals")
    vitals = pd.read_csv("data/mimic-iv-1.0/icu/chartevents.csv")
    vitals = vitals[vitals['itemid'].isin(vitals_itemids)]
    vitals = vitals[vitals['valuenum'] >= 0]
    vitals = vitals[vitals['valuenum'] < 9999]
    
    if verbose: print("Saving dfs")
    filtered_vitals_df = vitals.drop_duplicates()
    filtered_vitals_df.to_pickle('vitals')

    return filtered_vitals_df

'''
Gather urine output. 
'''
def gather_outputs(output_itemids, verbose=False):

    if verbose: print("Gathering outputs")
    outputs = pd.read_csv("data/mimic-iv-1.0/icu/outputevents.csv")
    ouptuts = outputs[outputs['itemid'].isin(output_itemids)]
    outputs = outputs[outputs['value'] >= 0]
    outputs = outputs[outputs['value'] < 99999]

    filtered_output_df = outputs

    if verbose: print("Saving dataframes")
    filtered_output_df.to_pickle('output')
    return filtered_output_df

'''
Gathers information about ICU level labs. 
'''
def gather_labs(labs_dict, verbose=False):
    labs_itemids = []
    for key in labs_dict:
        itemids = labs_dict[key]
        labs_itemids += itemids

    if verbose: print("Gathering labs")
    labs = pd.read_csv("data/mimic-iv-1.0/icu/chartevents.csv")
    labs = labs[labs['itemid'].isin(labs_itemids)]
    labs = labs[labs['valuenum'] >= 0]
    labs = labs[labs['valuenum'] < 99999]
    
    filtered_labs_df = labs.drop_duplicates()

    if verbose: print("Saving dataframes")
    filtered_labs_df.to_pickle('labs')
    return filtered_labs_df
'''
Gather ventilator procedures. 
'''
def gather_vent(vent_dict, verbose=False):
    vent_ids = []
    for key in vent_dict:
        vent_ids += vent_dict[key]
    
    if verbose: print("Gathering ventilator info")
    inputevents = pd.read_csv("data/mimic-iv-1.0/icu/inputevents.csv")
    vent = inputevents[inputevents['itemid'].isin(vent_ids)]
    vent = vent[vent['amount'] > 0]
    vent = vent[vent['amount'] < 99999]
    
    vent = vent.drop_duplicates()
    if verbose: print("Saving dataframes")
    vent.to_pickle('vent')
    return vent

# ChartFrames for both cohorts

In [ ]:
def buildTimeFrame(df_adm, delta=6):

    # Get admit and discharge time in numeric form, round down/up respectively to the nearest hour:
    start = pd.to_datetime(df_adm.intime.unique()).tolist()[0]
    start -= dt.timedelta(minutes=start.minute, seconds=start.second, microseconds=start.microsecond)

    end = pd.to_datetime(df_adm.outtime.unique()).tolist()[0]
    end -= dt.timedelta(minutes=end.minute, seconds=end.second, microseconds=end.microsecond)
    end += dt.timedelta(hours=1)

    times = []
    curr = start
    while curr < end:
        times.append(curr)
        curr += dt.timedelta(hours=delta)
    timeFrame = pd.DataFrame(data={'timestamp': times}, index=times)
    return timeFrame

'''
Build time-indexed dataframe of each patient admission, with resampled values of all variables.
hadm_id: hadm_id of the patient stay
tables: all of the tables generated in _load_data() 
popmeans: population level means used for imputation
verbose: flag for printing progress
timedelta: length in hours of the time interval for the rows
'''
def chartFrames(hadm_id, tables, verbose=False, timedelta=6):

    adm = 1; morb = 1; vits = 1; lbs = 1; mds = 1; prcs = 0; vnt = 1
    admissions, meds, vitals, labs, outputs, vent = tables

    df_adm = admissions[admissions.hadm_id == hadm_id].drop_duplicates()
    chart = buildTimeFrame(df_adm)

    if adm:
        if verbose: print('Admission Data')
        for var in ['stay_id', 'anchor_age', 'patientweight', 'los', 'gender']:
            chart[var.lower()] = df_adm[var].head(1).item()
        chart['gender'] = (chart['gender'] == 'F').astype(int)

    if morb:
        if verbose: print("Comorbidities")
        df_comorbs = comorbs[comorbs.hadm_id == hadm_id]
        for subpop in self.params.comorbidities_dict:
            subpop_df = df_comorbs[df_comorbs.long_title.isin(self.params.comorbidities_dict[subpop])]
            if subpop_df.empty:
                chart[subpop] = 0
            else:
                chart[subpop] = 1
        chart['expired'] = 0
    if lbs:
        if verbose: print('Lab tests')  # Using result date
        df_labs = labs[labs.hadm_id == hadm_id].drop_duplicates()

        # ICU Labs
        for k in sorted(list(pm.labs_dict_icu.keys())):
            chart[k.lower()] = np.nan
            for t in chart.timestamp:
                subset = df_labs[df_labs['itemid'].isin(pm.labs_dict_icu[k])]
                for i in subset.index:
                    if ((pd.to_datetime(subset.loc[i, 'charttime']) >= t) & (
                        pd.to_datetime(subset.loc[i, 'charttime']) < t + dt.timedelta(hours=timedelta))):
                        if k not in ['Ca', 'Glucose', 'CPK', 'PTH', 'LDH', 'AST', 'ALT']:
                            chart.loc[t, k.lower()] = np.nanmean([chart.loc[t, k.lower()], subset.loc[i, 'valuenum']])
                        else:
                            chart.loc[t, k.lower()] = 1

            chart[k.lower()] = chart[k.lower()].fillna(method='ffill', limit=3).fillna(value=0)

    if vits:
        if verbose: print('Vitals')
        df_vits = vitals[vitals.hadm_id == hadm_id]
        for k in sorted(list(pm.vitals_dict_icu.keys())):
            if k == "URINE_OUTPUT":
                continue
            chart[k.lower()] = np.nan
            for t in chart.timestamp:
                subset = df_vits[df_vits.itemid.isin(pm.vitals_dict_icu[k])]
                for i in subset.index:
                    try:
                        if ((pd.to_datetime(subset.loc[i, 'charttime']) >= t) & (
                            pd.to_datetime(subset.loc[i, 'charttime']) < t + dt.timedelta(hours=timedelta))):
                            chart.loc[t, k.lower()] = np.nanmean([chart.loc[t, k.lower()], subset.loc[i, 'value']])
                    except:
                        try:
                            if ((pd.to_datetime(subset.loc[i, 'charttime']) >= t).any() & (
                                pd.to_datetime(subset.loc[i, 'charttime']) < t + dt.timedelta(hours=timedelta)).any()):
                                chart.loc[t, k.lower()] = np.nanmean([chart.loc[t, k.lower()], subset.loc[i, 'value']])
                        except:
                            pass
            chart[k.lower()] = chart[k.lower()].fillna(method='ffill').fillna(value=0)


    if mds:
        if verbose: print('Medications')
        df_meds = meds[meds.hadm_id==hadm_id].drop_duplicates()
        for k in sorted(list(pm.meds_dict_icu.keys())):
            chart[k.lower()] = 0
            subset = df_meds[df_meds.itemid.isin(pm.meds_dict_icu[k])]
            for t in chart.timestamp:
                for i, row in subset.iterrows():
                    if row.amountuom == 'dose':
                        continue
                    if ((pd.to_datetime(row.starttime) >= t) and (pd.to_datetime(row.starttime) <  t + dt.timedelta(hours=timedelta))):
                        if k in ['K-IV', 'K-nonIV', 'Mg-IV', 'Mg-nonIV', 'P-IV', 'P-nonIV']:
                            scaler = 1
                            if row.amountuom == 'grams':
                                scaler = 1000
                            elif row.amountuom == 'mcg':
                                scaler = 0.001
                            elif row.amountuom == 'pg':
                                scaler = 1e-9
                            elif row.amountuom == 'ml':
                                scaler = 1
                            chart.loc[t, k.lower()] += scaler * float(row.amount)
                            td = pd.to_datetime(row.endtime) - pd.to_datetime(row.starttime)
                            hours = td.days * 24 + td.seconds // 3600
                            chart.loc[t, 'hours-'+k.lower()] = hours
                        else:
                            chart.loc[t, k.lower()] += 1
                    if row.endtime is not np.nan:
                        if ((pd.to_datetime(row.starttime) <= t) and (pd.to_datetime(row.endtime) > t)):
                            if k in ['K-IV', 'K-nonIV', 'Mg-IV', 'Mg-nonIV', 'P-IV', 'P-nonIV']:
                                scaler = 1
                                if row.amountuom == 'grams':
                                    scaler = 1000
                                elif row.amountuom == 'mcg':
                                    scaler = 0.001
                                elif row.amountuom == 'pg':
                                    scaler = 1e-9
                                elif row.amountuom == 'ml':
                                    scaler = 1
                                chart.loc[t, k.lower()] += scaler * float(row.amount)
                                td = pd.to_datetime(row.endtime) - pd.to_datetime(row.starttime)
                                hours = td.days * 24 + td.seconds // 3600
                                chart.loc[t, 'hours-'+k.lower()] = hours
                            else:
                                chart.loc[t, k.lower()] += 1
    if vnt:
        if verbose: print('Ventilation')
        df_vent = vent[vent.hadm_id == hadm_id].drop_duplicates()
        if df_vent.empty:
            chart['vent'] = 0
        else:
            chart['vent'] = np.nan
            for t in chart.timestamp:
                for i in df_adm.index:
                    if ((pd.to_datetime(df_vent.loc[i, 'charttime']) >= t) and (
                        pd.to_datetime(df_vent.loc[i, 'charttime']) < t + dt.timedelta(hours=6))):
                        chart.loc[t, 'vent'] = np.nanmean([chart.loc[t, 'vent'], df_vent.loc[i, 'value']])
                    else:
                        chart.loc[t, 'vent'] = 0

    chart = chart[~np.isnat(chart.timestamp)]
    chart = chart.dropna()
    if verbose: print('Done!')

    return chart

In [ ]:
def gridBatch(patient_ids, tables, name):
    batchCharts = pd.DataFrame()

    for i, vnum in tqdm.tqdm(enumerate(patient_ids)):
        #try:
        chart = chartFrames(vnum, tables, verbose=False)
        batchCharts = batchCharts.append(chart, ignore_index=True)
        if i % 50 == 0:
            batchCharts.to_csv(str(i) + '_' + str(name) + '_checkpoint.csv', index=False)
        #except:
        #    print("Issue in : " + str(vnum))
    print('Batch done!')
    batchCharts.to_csv(name + '_allFrames.csv', index=False)

In [ ]:
def demographic_info(hadm_ids):    
    admissions = pd.read_csv("data/mimic-iv-1.0/core/admissions.csv")
    admissions = admissions[admissions['hadm_id'].isin(hadm_ids)]
    admissions = admissions[['hadm_id', 'insurance', 'marital_status', 'ethnicity']]
        
    return admissions

# Generate tabular data

In [ ]:
# Run this cell if this is the first time
meds = gather_meds(pm.meds_dict_icu, verbose=True)
vitals = gather_vitals(pm.vitals_dict_icu, verbose=True)
outputs = gather_outputs(pm.vitals_dict_icu['Urine'], verbose=True)
labs = gather_labs(pm.labs_dict_icu, verbose=True)
vent = gather_vent(pm.vent_dict, verbose=True)

In [ ]:
# Run this cell if you have saved dataframes
meds = pickle.load(open('meds', 'rb'))
vitals = pickle.load(open('vitals', 'rb'))
outputs = pickle.load(open('outputs', 'rb'))
labs = pickle.load(open('labs', 'rb'))
vent = pickle.load(open('vent', 'rb'))

## Melanoma

In [ ]:
diagnoses_icd = pd.read_csv("data/mimic-iv-1.0/hosp/d_icd_diagnoses.csv")
melanoma_codes = diagnoses_icd[diagnoses_icd['long_title'].str.contains('melanoma')]
melanoma = []
for c, v in zip(melanoma_codes['icd_code'], melanoma_codes['icd_version']):
    melanoma.append((c, v))
hadm_ids, admissions = gather_cohort(icd_diagnoses=melanoma, verbose=True)
demographics = demographic_info(hadm_ids)

In [ ]:
gridBatch(hadm_ids, (admissions, meds, vitals, labs, outputs, vent), 'melanoma')

## Heart Disease

In [ ]:
cad_names = ['Atherosclerotic heart disease of native coronary artery without angina pectoris',
'Atherosclerotic heart disease of native coronary artery with unstable angina pectoris', 
'Atherosclerotic heart disease of native coronary artery with angina pectoris with documented spasm', 
'Atherosclerotic heart disease of native coronary artery with other forms of angina pectoris', 
'Atherosclerotic heart disease of native coronary artery with unspecified angina pectoris']
diagnoses_icd = pd.read_csv("data/mimic-iv-1.0/hosp/d_icd_diagnoses.csv")
hd_codes = diagnoses_icd[diagnoses_icd['long_title'].isin(cad_names)]
hd = []
for c, v in zip(hd_codes['icd_code'], hd_codes['icd_version']):
    hd.append((c, v))
hadm_ids, admissions = gather_cohort(icd_diagnoses=hd, verbose=True)
demographics = demographic_info(hadm_ids)

In [ ]:
gridBatch(hadm_ids, (admissions, meds, vitals, labs, outputs, vent), 'hd')

## Flu/Pneumonia

In [ ]:
fp_names = []
diagnoses_icd = pd.read_csv("data/mimic-iv-1.0/hosp/d_icd_diagnoses.csv")
all_diag = diagnoses_icd['long_title']
for d in all_diag:
    if "pneumonia" in d.lower():
        fp_names.append(d)
    if "influenz" in d.lower():
        fp_names.append(d)
fp_names = list(set(fp_names))
hd_codes = diagnoses_icd[diagnoses_icd['long_title'].isin(fp_names)]
hd = []
for c, v in zip(hd_codes['icd_code'], hd_codes['icd_version']):
    hd.append((c, v))
hadm_ids, admissions = gather_cohort(icd_diagnoses=hd, verbose=True)
demographics = demographic_info(hadm_ids)

In [ ]:
gridBatch(hadm_ids, (admissions, meds, vitals, labs, outputs, vent), 'hd')

## Kidney Disease

In [ ]:
diagnoses_icd = pd.read_csv("data/mimic-iv-1.0/hosp/d_icd_diagnoses.csv")
diag_codes = diagnoses_icd[diagnoses_icd['long_title'].str.contains('end stage renal disease')]
rd_codes = diag_codes
hd = []
for c, v in zip(rd_codes['icd_code'], rd_codes['icd_version']):
    hd.append((c, v))
procedures_desc = pd.read_csv('data/mimic-iv-1.0/hosp/d_icd_procedures.csv')
procedures_df = pd.read_csv('data/mimic-iv-1.0/hosp/procedures_icd.csv')
procedures_desc['long_title'] = procedures_desc['long_title'].str.lower()
# last 19 items are related to imaging procedures so ignore them
transplant_procedure_codes = procedures_desc[procedures_desc['long_title'].str.contains('transplant')]
for c, v in zip(transplant_procedure_codes['icd_code'], transplant_procedure_codes['icd_version']):
    hd.append((c, v))

hadm_ids, admissions = gather_cohort(icd_diagnoses=hd, verbose=True)
demographics = demographic_info(hadm_ids)